In [ ]:
!pip install mne
!pip install plotly==5.3.1
!pip install pymatreader 
!pip install librosa
!pip install yasa
!pip install protobuf==3.19
!pip install --upgrade pip

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import glob

from scipy.io import wavfile, savemat
from scipy.fft import fft, fftfreq
from scipy import signal
from scipy.fft import fftshift
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
from IPython.display import clear_output

import librosa
import mne
import pymatreader
import yasa
import seaborn as sns
import pyxdf

from mne.time_frequency import psd_array_multitaper
from mne.decoding import CSP
from scipy.integrate import simps
from yasa import sliding_window

import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# CNN packages
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD,RMSprop,Adam
from keras.utils import np_utils
from keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [ ]:
# Set gpu as backend
import tensorflow as tf
import keras
config = tf.compat.v1.ConfigProto()
sess = tf.compat.v1.Session(config=config)
keras.backend.set_session(sess)

In [ ]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.preprocessing import StandardScaler

# Importing preprocessed data

In [ ]:
def get_trials():
    trial_ctr_dict = {}
    for x in events[1].keys():
        if "ActionBeg" in x:
            trial_ctr_dict[x[10:]] = 0
    return trial_ctr_dict

In [ ]:
def get_event_info(marker_stream):
    # Get audio info
    event_time = {"Name": [], "StartTime": [], "EndTime": [], "handPos": [], "Hand": [], "Action": []}
    for idx in range(len(marker_stream[0])):
        marker_list = marker_stream[0][idx][2]
        marker = list(marker_stream[1].keys())[list(marker_stream[1].values()).index(marker_list)]
        if "ActionBeg" in marker:
            start_time = marker_stream[0][idx][0]
            end_time = marker_stream[0][idx+1][0]
            action_name = marker[10:]
            event_time["Name"].append(action_name)
            event_time["StartTime"].append(start_time)
            event_time["EndTime"].append(end_time)
            event_time["handPos"].append(action_name.split("-")[0])
            event_time["Hand"].append(action_name.split("-")[1])
            event_time["Action"].append(action_name.split("-")[2])
    return event_time

In [ ]:
def save_time_series_as_df(event_time, eeg_data, file_name):
    eeg_df = []
    channels = np.array([[x] for x in arr.ch_names])
    trial_ctr_dict = get_trials()
    for ts in range(len(event_time["Name"])):
        p_eeg  = eeg_data.iloc[event_time["StartTime"][ts] - 25: event_time["StartTime"][ts] + 500, 1:].T
        handPos = np.array([[event_time["handPos"][ts]]] * 64)
        Hand = np.array([[event_time["Hand"][ts]]] * 64)
        Action = np.array([[event_time["Action"][ts]]] * 64)
        Subject = np.array([[file_name[:-4]]] * 64)
        trial_type = event_time["handPos"][ts] + "-" + event_time["Hand"][ts] + "-" + event_time["Action"][ts]
        trial_ctr_dict[trial_type] += 1
        Trial = np.array([[trial_ctr_dict[trial_type]]] * 64)
        
        # Apply baseline correction
        p_eeg.iloc[:, :] = mne.baseline.rescale(p_eeg.to_numpy(), np.array(list(range(525))), baseline=(0, 25), mode="zscore")

        p_eeg = np.append(p_eeg, handPos, axis=1)
        p_eeg = np.append(p_eeg, Hand, axis=1)
        p_eeg = np.append(p_eeg, Action, axis=1)
        p_eeg = np.append(p_eeg, Subject, axis=1)
        p_eeg = np.append(p_eeg, channels, axis=1)
        p_eeg = np.append(p_eeg, Trial, axis=1)
        
        
        if len(eeg_df) == 0:
            eeg_df = p_eeg
        else: 
            try:
                eeg_df = np.concatenate((eeg_df, p_eeg), axis=0)
            except:
                pass
        print(eeg_df.shape)

    return eeg_df 

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessedv2/")
print("Current directory: ", os.getcwd())
data_files = glob.glob("*.set")
print("Files in current directory: ", data_files)
eeg_df_main = []
for file_name in data_files:
    print("Current file: ", file_name)
    arr = mne.io.read_raw_eeglab(file_name)
    eeg_data = arr.to_data_frame()
    events = mne.events_from_annotations(arr)
    event_time = get_event_info(events)
    eeg_df = save_time_series_as_df(event_time, eeg_data, file_name)
    
    if len(eeg_df_main) == 0:
        eeg_df_main = eeg_df
    else: 
        eeg_df_main = np.concatenate((eeg_df_main, eeg_df), axis=0)
    print(eeg_df_main.shape)

In [ ]:
# row: 460800
df = pd.DataFrame(eeg_df_main, columns=list(range(-100, 2000, 4))+["handPos", "Hand", "Action", "Subject", "Channel", "Trial"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
df.to_csv("cleaned_datav2.csv", index=False, header=list(range(-100, 2000, 4))+["handPos", "Hand", "Action", "Subject", "Channel", "Trial"])
df

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2.csv")
dataset

In [ ]:
dataset.iloc[:, :-6]

In [ ]:
list(np.unique(list(dataset.iloc[:, -6])))

In [ ]:
def save_time_series_mat(dataset):
    subjects = list(np.unique(list(dataset.iloc[:, -3])))
    conditions = list(np.unique(list(dataset.iloc[:, -6])))
    for sub in subjects:
        for palm in conditions:
            for oc in ["Open", "Close"]:
                for hand in ["Left", "Right"]:
                    for i in range(40):
                        test_sub_trial = dataset[
                            (dataset["handPos"] == palm) & 
                            (dataset["Action"] == oc) & 
                            (dataset["Hand"] == hand) & 
                            (dataset["Trial"] == i+1) & 
                            (dataset["Subject"] == sub)].iloc[:, :-6]
                        if test_sub_trial.shape != (525, 64):
                            continue
                        test_sub_trial.index = test_sub_trial.iloc[:, -2]
                        savename = str(sub + "_" + palm + "_" + oc + "_" + hand + "_" + str(i+1))
                        savemat("Matv1/" + savename+".mat", mdict={"EEG": test_sub_trial.to_numpy() })        

os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Matlab")
save_time_series_mat(dataset)

In [ ]:
# def save_time_series_mat(audio_time, eeg_data, file_name):
#     for ts in range(len(audio_time["Name"])):
#         p_eeg  = eeg_data.iloc[audio_time["StartTime"][ts]: audio_time["EndTime"][ts], 1:]
#         save_name = file_name.split(".")[0] + "_" + str(audio_time["Name"][ts]) + "_" + str(audio_time["Trial"][ts]) + "_" + str(audio_time["Rating"][ts]) + ".mat"
#         save_name = save_name.replace("-", "_")
#         savemat("mat/" + save_name, mdict={"EEG": p_eeg.to_numpy() })

# Common Spatial Patterns

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2.csv")
dataset

In [ ]:
# 64 channel EEG data with 500 samples
# Giving us X = N * 64 * 275 and y = N 
X, y = [], []
nchan = 64
new_dataset = dataset[~(dataset["Subject"] == "P01")]
iterations = len(dataset) // nchan
for idx in range(iterations):
    start = idx * nchan
    end = (idx + 1) * nchan

    X.append(np.expand_dims(dataset.iloc[start: end, :-6], 2))
    y.append(dataset.iloc[start, -6])
#     print(len(X), len(y))
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
yl = le.transform(y)
yl

In [ ]:
X.shape, y.shape, yl.shape

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[:, :, 60:124, 0], yl, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
# X_red = np.array([x.T for x in X[:, :, :, 0]])
X_red = X[:, :, :, 0]
X_red.shape

In [ ]:
selected_channels = [6, 20, 28, 34, 35, 37, 39, 42, 50, 60]
X_int = X[:, :, 25: 200 , :]
X_int = X_int[:, :, :, 0]
labels = yl
X_int.shape, labels.shape

In [ ]:
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(X)

sfreq = 250
w_length = int(sfreq * 0.5)   # running classifier: window length
w_step = int(sfreq * 0.1)  # running classifier: window step size
w_start = np.arange(0, X_int.shape[2] - w_length, w_step)
print(w_start)
scores_windows = []

# Assemble a classifier
scores = []
clf = SVC(gamma='auto') #make_pipeline(StandardScaler(), SVC(gamma='auto'))
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]
    X_train = csp.fit_transform(X_int[train_idx], y_train)
    X_test = csp.transform(X_int[test_idx])
    print(X_train.shape, X_test.shape)
    plt.scatter(X_train, y_train)
    
#     # Wavelet decomposition
#     X_train_feat = np.array(mne.time_frequency.morlet(250, [40], n_cycles=7)[0].imag / 20).reshape(1, -1)
#     X_test_feat = np.array(mne.time_frequency.morlet(250, [40], n_cycles=7)[0].imag / 20).reshape(1, -1)
    
#     print(X_test_feat.shape)
# #     print(X_train_feat[0].shape, len(X_test_feat))
    
#     # fit classifier
# #     lda.fit(X_train, y_train)
#     clf.fit(X_train_feat, y_train)
    
#     # running classifier: test classifier on sliding window
#     score_this_window = []
#     for n in w_start:
#         X_test = csp.transform(X_int[test_idx][:, :, n:(n + w_length)])
#         score_this_window.append(clf.score(X_test_feat, y_test))
#     scores_windows.append(score_this_window)
    

In [ ]:
scores_windows

In [ ]:
X_train[:, [2, 3]].shape

In [ ]:
plt.scatter(X_train[y_train==0,0], X_train[y_train==0,1])
plt.scatter(X_train[y_train==1,0], X_train[y_train==1,1])
plt.show()

In [ ]:
# https://github.com/spolsley/common-spatial-patterns
# Common Spatial Pattern implementation in Python, used to build spatial filters for identifying task-related activity.
import numpy as np
import scipy.linalg as la

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)

In [ ]:
csp = CSP(X_red[yl==0], X_red[yl==1])

In [ ]:
csp[0].shape

In [ ]:
csp[0] - csp[1]

# ERP Analysis

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2_1.csv")
dataset

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessedv2/")
arr = mne.io.read_raw_eeglab("P01.set")
#     eeg_data = arr.to_data_frame()
channels = arr.ch_names#np.array([[x] for x in ])
events = mne.events_from_annotations(arr)
channels

In [ ]:
test_sub_trial = dataset[(dataset["handPos"] == "palmDown") & (dataset["Action"] == "Open") & (dataset["Hand"] == "Right") & (dataset["Trial"] == 1) & (dataset["Subject"] == "P01")]
test_sub_trial

In [ ]:
conditions_dict = {}
conds_trial_ctr = {}
for x in events[1].keys():
    if "ActionBeg" in x:
        conditions_dict[x[10:]] = []
        conds_trial_ctr[x[10:]] = 0
conditions_dict, conds_trial_ctr

In [ ]:
for subs in ["P01", "P02", "P03", "P04"]:
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                for i in range(40):
                    test_sub_trial = dataset[
                        (dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Trial"] == i+1) & 
                        (dataset["Subject"] == subs)]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    name = palm + "-" + hand + "-" + oc 
                    if len(conditions_dict[name]) > 0:
                        try:
                            conditions_dict[name] += test_sub_trial.iloc[:, :-6].to_numpy()
                        except:
                            pass
                    else:
                        conditions_dict[name] = test_sub_trial.iloc[:, :-6].to_numpy()

                    conds_trial_ctr[name] += 1
                    (name, test_sub_trial.shape, test_sub_trial)

In [ ]:
conds_trial_ctr, conditions_dict

In [ ]:
cond_dict_avg = {}
for key in conditions_dict.keys():
    cond_dict_avg[key] = conditions_dict[key] / conds_trial_ctr[key]
cond_dict_avg

In [ ]:
int_chan = 6
for hand in ["Right"]:
    for handPos in ["palmDown", "palmIn", "palmUp"]:
        lines = []
        posture = ""
        
        for key in cond_dict_avg.keys():
            if handPos in key and hand in key:
                lines.append(np.array(cond_dict_avg[key][int_chan, :200]))
                posture += str(key.split("-")[1:])
        
        fig = px.line(
                np.array(lines).T, 
                title=handPos + " " + str(int_chan) + " " + hand, 
        )
        print(posture)
        fig.show()


In [ ]:
# Plotting difference wave
int_chan = 20
for hand in ["Left", "Right"]:
    for handPos in ["palmDown", "palmIn", "palmUp"]:
        print(handPos + "-" + hand)
        
        open_arr = cond_dict_avg[handPos + "-" + hand + "-" + "Open"][int_chan, :200]
        close_arr = cond_dict_avg[handPos + "-" + hand + "-" + "Close"][int_chan, :200]
        
        diff = np.expand_dims(np.array(open_arr - close_arr), 1)
        final = diff
        
#         final = np.append(final, np.expand_dims(open_arr, axis=1), axis=1)
#         final = np.append(final, np.expand_dims(close_arr, axis=1), axis=1)
        print(final.shape)
        
        fig = px.line(
                final, 
                title=handPos + " " + str(int_chan) + " " + hand, 
        )
        fig.show()
        

In [ ]:
int_chan = 42
for hand in ["Left", "Right"]:
    for handPos in ["palmDown", "palmIn", "palmUp"]:
        lines = []
        posture = ""
        for key in cond_dict_avg.keys():
            if handPos in key and hand in key:
                lines.append(np.array(cond_dict_avg[key][int_chan, :175]))
                posture += str(key.split("-")[1:])
        fig = px.line(
                np.array(lines).T, 
                title=handPos + " " + str(int_chan) + " " + hand, 
            )
        print(posture)
        fig.show()


In [ ]:
req_chans = [[12, 6, 60], [20, 50], [28, 34,42], [35, 39, 37]]
for handPos in ["palmDown", "palmIn", "palmUp"]:
    plot_lines = []
    for chans in req_chans:
        for key in cond_dict_avg.keys():
            if handPos in key:
                fig = px.line(
                    cond_dict_avg[key][chans, :175].T, 
                    title=key + " " + str(chans), 
                )
                fig.show()
        

In [ ]:
cond_dict_avg["palmDown-Left-Close"].shape

In [ ]:
exclusions = [0, 4, 16]
channels = [i if i not in exclusions else -1 for i in list(range(64))]
for key in cond_dict_avg.keys():
    fig = px.line(
        cond_dict_avg[key][:, :175].T, 
        title=key, 
    )
    fig.show()

In [ ]:
# Averaging over all hand positions. Taking only right hand open/close
handPos = ["palmDown", "palmIn", "palmUp"]
oc_erp = {"Open": [], "Close": []}
for key in cond_dict_avg.keys():
#     if "Right" in key: 
    if "Open" in key:
        oc_erp["Open"] = cond_dict_avg[key] if len(oc_erp["Open"]) == 0 else oc_erp["Open"] + cond_dict_avg[key]
    if "Close" in key:
        oc_erp["Close"] = cond_dict_avg[key] if len(oc_erp["Close"]) == 0 else oc_erp["Close"] + cond_dict_avg[key]

oc_erp

In [ ]:
oc_erp_avg = {}
for key in oc_erp.keys():
    oc_erp_avg[key] = oc_erp[key] / 6
oc_erp_avg

In [ ]:
for key in oc_erp_avg.keys():
    fig = px.line(
        oc_erp_avg[key][:, :175].T, 
        title=key, 
    )
    fig.show()

# Importing Data

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_datav2.csv")
dataset

In [ ]:
# 64 channel EEG data with 500 samples
# Giving us X = N * 64 * 275 and y = N 
X, y = [], []
nchan = 64
new_dataset = dataset[~(dataset["Subject"] == "P01")]
iterations = len(dataset) // nchan
for idx in range(iterations):
    start = idx * nchan
    end = (idx + 1) * nchan

    X.append(np.expand_dims(dataset.iloc[start: end, :-6], 2))
    y.append(dataset.iloc[start, -6])
#     print(len(X), len(y))
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
y

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
yl = le.transform(y)
yl

In [ ]:
np.unique(yl)

# Visualizing Data

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Plots/RawV5/")
for sub in ["P01", "P02", "P03", "P04"]:
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                for i in range(40):
                    test_sub_trial = dataset[
                        (dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Trial"] == i+1) & 
                        (dataset["Subject"] == sub)]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    fig = px.line(
                        test_sub_trial.iloc[:, :-6].T, 
                        title=str(sub + palm + oc + hand + str(i)), 
                        labels=test_sub_trial.iloc[:, -2]
                    )
                    fig.write_image(sub + palm + oc + hand + str(i)+ ".png") 
#                     fig.show()

In [ ]:
dataset[
        (dataset["handPos"] == "palmIn") & 
        (dataset["Action"] == "Open") & 
        (dataset["Hand"] == "Right") & 
        (dataset["Trial"] == 1) & 
        (dataset["Subject"] == "P01")].iloc[:, :]

In [ ]:
for i in range(40):
    test_sub_trial = dataset[
        (dataset["handPos"] == "palmIn") & 
        (dataset["Action"] == "Open") & 
        (dataset["Hand"] == "Right") & 
        (dataset["Trial"] == i+1) & 
        (dataset["Subject"] == "P01")]
    test_sub_trial.index = test_sub_trial.iloc[:, -2]
    fig = px.line(
        test_sub_trial.iloc[:, :150].T, 
        #title=str(sub + palm + oc + hand + str(i)), 
        title="Right",
        labels=test_sub_trial.iloc[:, -2]
    )
    #fig.write_image(sub + palm + oc + hand + str(i)+ ".png") 
    fig.show()
    test_sub_trial = dataset[
        (dataset["handPos"] == "palmIn") & 
        (dataset["Action"] == "Open") & 
        (dataset["Hand"] == "Left") & 
        (dataset["Trial"] == i+1) & 
        (dataset["Subject"] == "P01")]
    test_sub_trial.index = test_sub_trial.iloc[:, -2]
    fig = px.line(
        test_sub_trial.iloc[:, :150].T, 
        #title=str(sub + palm + oc + hand + str(i)), 
        title="Left",
        labels=test_sub_trial.iloc[:, -2]
    )
    #fig.write_image(sub + palm + oc + hand + str(i)+ ".png") 
    fig.show()

# Classification

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[:, :, 60:124], yl, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
#https://github.com/vlawhern/arl-eegmodels
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 125, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(1, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('sigmoid', name = 'sigmoid')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model = EEGNet(2, 64, 64)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

# count number of parameters in the model
numParams = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpoint_filepath = "/tmp/checkpoint/"
checkpointer = ModelCheckpoint(
        filepath=checkpoint_filepath, 
        save_weights_only=True,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True
)

# Handle class imbalance
class_weights = {0:1, 1:1}

In [ ]:
################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
batch_size = 512
EPOCHS = 100
history = model.fit(
    X_train, 
    y_train, 
    batch_size = batch_size, 
    epochs = EPOCHS, 
    validation_data=(X_eval, y_eval),
    callbacks=[checkpointer], 
#     class_weight = class_weights
)

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
model.load_weights(checkpoint_filepath)
model.save('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Models/model-eegnet-v2')

In [ ]:
new_model = tf.keras.models.load_model('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Models/model-eegnet-v2')

y_pred = np.argmax(new_model.predict(X_test), axis=1)
y_true = y_test
test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy main: {test_acc:.0%}')

In [ ]:
print("Max train accuracy: ", max(history.history["accuracy"]))
print("Max validation accuracy: ", max(history.history["val_accuracy"]))

In [ ]:
# Multitaper method
def bandpower_multitaper(data, sf, method, band, relative=False):
    band = np.asarray(band)
    low, high = band

    if method == 'multitaper':
        psd_trial, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                                normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd_trial[idx_band], dx=freq_res)

    return bp


In [ ]:
def create_bandpower_features(epochs, frequency):
    bandpower_multitaper_EEG = []
    # Iterating over each subject [20]
    for epoch in epochs:
        # Iterating over each song per subject [30]
        bands_video = []
        no_channels = epoch.shape[0]
        input_brainwaves = epoch
        # Iterating over each channel [14]
        for k in range(no_channels):
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[0.1, 4], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[4, 7], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 13], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[14, 30], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[31, 50], relative=False))
        
#         print(len(bands_video), bands_video[-1].shape, bands_video)
        
        bandpower_multitaper_EEG.append(bands_video)

    bandpower_multitaper_EEG = np.array(bandpower_multitaper_EEG)
  
    return bandpower_multitaper_EEG

In [ ]:
sf = 250
window = 0.5
psd_features = []
X_req =  X[:, :, 35:161, 0]
for x in X_req:
    epoch = sliding_window(x, sf=sf, window=window, step=window)[1]
    psd = create_bandpower_features(epoch, sf)
    if len(psd_features) == 0:
        psd_features = psd
    else:
        psd_features = np.concatenate((psd_features, psd))
    print(psd_features.shape)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data')
np.save("IntData/psd_features_v2_2.npy", psd_features)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
psd_labels = []
for y in yl:
    psd_labels += [y] * 1
np.save("psd_labels_v2_2.npy", psd_labels)

In [ ]:
np.unique(yl)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
psd_labels = []
for y in yl:
    psd_labels += [y] * 1
np.save("psd_labels_pos_v2_2.npy", psd_labels)

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("psd_features_v2_2.npy"), np.load("psd_labels_v2_2.npy")
X.shape, y.shape # ((1920, 320), (1920,))

In [ ]:
X[:, 3:5].shape

In [ ]:
# With select channels
# selected_channels = [6, 20, 28, 34, 35, 37, 39, 42, 50, 60]
X_sel = []
for i in range(64):
    start = i * 5
    end = (i+1) * 5
    if i in selected_channels:
        if len(X_sel) == 0:
            X_sel = X[:, start:end]
        else:
            X_sel = np.concatenate((X_sel, X[:, start:end]), axis=1)

X_sel = np.array(X_sel)
X_sel.shape, X_sel

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[:480, :], y[:480], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
np.unique(y_train)

In [ ]:
# source : https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

clfs = [
    KNeighborsClassifier(2),
    # SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=2000),
    RandomForestClassifier(max_depth=2000, n_estimators=100, max_features=1),
    MLPClassifier(alpha=1, max_iter=2000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

names = [
    "Nearest Neighbors",
    # "Linear SVM",
    "RBF SVM",
    # "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]


In [ ]:
results = []
for idx,clf in enumerate(clfs):
    clf.fit(X_train[:, [3, 4]], y_train)
    score = clf.score(X_test[:, [3, 4]], y_test)
    results.append([names[idx], score])
    print(names[idx], score)

# Time-Frequency

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Matlab/spectopov1/")
processed = glob.glob("*.mat")
len(processed), processed

In [ ]:
data = loadmat("P03_palmIn_Close_Left_13_spec.mat")["spectra"]
data.shape

In [ ]:
grasp_dict = {"Open": 0, "Close": 1}

In [ ]:
X, y = [], []
for mat in processed:
    data = loadmat(mat)["spectra"]
    data = np.expand_dims(data, 3)
    print(data.shape)
    if len(X) > 0:
        X = np.concatenate((X, data))
    else:
        X = data

    label = grasp_dict[mat.split("_")[2]]
    y += [label] * data.shape[0] 

    print(X.shape, len(y))
    assert(X.shape[0] == len(y)), mat

In [ ]:
X.shape, len(y)

In [ ]:
for i in range(10):
    idx = np.random.randint(10080)
    plt.pcolormesh(X[idx][:, :,0])
    print(y[idx])
    plt.show()

In [ ]:
y = np.array(y)
y.shape

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X[:, :, 24:, :], y[:], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
# X_eval, X_train, y_eval, y_train = train_test_split(X, y, test_size=0.2, random_state=42)
# X_test, X_eval, y_test, y_eval = train_test_split(X_eval, y_eval, test_size=0.2, random_state=42)

# X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
def create_checkpoint():
    checkpoint_filepath = '/tmp/checkpoint/'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    return checkpoint_filepath, model_checkpoint_callback

checkpoint_filepath, model_checkpoint_callback = create_checkpoint()
checkpoint_filepath

In [ ]:
def create_model():
    model = Sequential()
    
    model.add(Convolution2D(32,(3, 3), input_shape=(64, 41, 1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(64,(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(256,(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
#     model.add(Convolution2D(512,(3, 3), activation="relu"))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  
#     model.add(Dense(1024, activation="relu"))
#     model.add(Dropout(.5))
    
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(.5))
    
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
    
    model.summary()
    return model

model = create_model()

In [ ]:
batch_size = 512
EPOCHS = 50
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=EPOCHS,
    validation_data=(X_eval, y_eval),
    callbacks=[model_checkpoint_callback],
    verbose=1
)

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()

In [ ]:
model.load_weights(checkpoint_filepath)
# model.save('/mnt/sda1/shivam/DrummingEEG/FullStudyData/Models/model-v2')

In [ ]:
# new_model = tf.keras.models.load_model('/mnt/sda1/shivam/DrummingEEG/FullStudyData/Models/model-v2')

y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = y_test
test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy main: {test_acc:.0%}')

# Dimensionality Reduction

# t-SNE

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("psd_features_v2_2.npy"), np.load("psd_labels_pos_v2_2.npy")
X.shape, y.shape # ((1920, 320), (1920,))

In [ ]:
dataset_embedded = TSNE(
        n_components=3, 
        learning_rate="auto", 
        init="random", 
        perplexity=8
    ).fit_transform(X)

In [ ]:
dataset_embedded.shape

In [ ]:
fig = px.scatter_3d(dataset_embedded, x=0, y=1, z=2, color=0)
fig.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
perplexity = range(0, 41, 4)
for per in perplexity:
    dataset_embedded = TSNE(
        n_components=2, 
        learning_rate="auto", 
        init="random", 
        perplexity=per
    ).fit_transform(X)
    
    print("Perplexity: ", per)
    plt.scatter(dataset_embedded[y==0,0], dataset_embedded[y==0,1])
    plt.scatter(dataset_embedded[y==1,0], dataset_embedded[y==1,1])
#     plt.scatter(dataset_embedded[y==2,0], dataset_embedded[y==2,1])
    plt.show()

# PCA

In [ ]:
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("psd_features_v2_2.npy"), np.load("psd_labels_v2_2.npy")
X.shape, y.shape # ((1920, 320), (1920,))

In [ ]:
np.unique(y)

In [ ]:
list(range(10, 320, 10))

In [ ]:
# PCA
from sklearn.decomposition import PCA
feature_size = range(320, 10, 10)
for n_comp in list(range(10, 320, 10)):
    pca = PCA(n_components=n_comp)
    X_r = pca.fit(X).transform(X)
    X_train, X_eval, y_train, y_eval = train_test_split(X_r, y, test_size=0.15, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    print("NUMBER OF COMPONENTS: ", n_comp)
    for idx,clf in enumerate(clfs):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print("\t", names[idx], score)

In [ ]:
# X_train, X_eval, y_train, y_eval = train_test_split(X_r, y, test_size=0.15, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
# X_train.shape, X_eval.shape, X_test.shape, y_train.shape, y_eval.shape, y_test.shape

In [ ]:
results = []
for idx,clf in enumerate(clfs):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    results.append([names[idx], score])
    print(names[idx], score)

# SVD

In [ ]:
# truncatedSVD iterative
from sklearn.decomposition import TruncatedSVD
os.chdir('/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/')
X, y = np.load("psd_features_v3_1.npy"), np.load("psd_labels_v3_1.npy")

feature_size = range(320, 10, 10)
for n_components in list(range(10, 320, 10)):
    
    svd = TruncatedSVD(n_components, n_iter=7, random_state=42)
    X_r = svd.fit(X).transform(X)
    
    X_train, X_eval, y_train, y_eval = train_test_split(X_r, y, test_size=0.15, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    
    print("NUMBER OF COMPONENTS: ", n_components)
    for idx,clf in enumerate(clfs):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print("\t", names[idx], score)